<a href="https://colab.research.google.com/github/canabica/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from numba import jit, prange
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import kurtosis, skew

# Configurações de constantes e parâmetros
DRIVE_MOUNT_PATH = "/content/drive"
GRAPHPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive/graficos")
SUMMARYPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive/sumarios")
PALETTE = sns.color_palette("Set2")
FIGSIZE = (10, 6)

# Função para carregar dados CSV
def load_data(filepath, delimiter=',', encoding='utf-8'):
    try:
        df = pd.read_csv(filepath, delimiter=delimiter, encoding=encoding)
        return dd.from_pandas(df, npartitions=4)
    except FileNotFoundError:
        print(f"Arquivo não encontrado: {filepath}")
        return None

# Função para salvar gráficos
def save_fig(filename, graphpath):
    plt.savefig(os.path.join(graphpath, filename), dpi=300, bbox_inches='tight')
    plt.close()

# Função para gerar gráficos de violino
def generate_violinplot(df, x, y, hue, title, filename, graphpath, split=False):
    plt.figure(figsize=FIGSIZE)
    try:
        sns.violinplot(x=x, y=y, hue=hue, data=df.compute(), split=split, palette=PALETTE)
    except ValueError as e:
        print(f"Erro ao gerar gráfico de violino: {e}")
        return
    plt.title(title)
    save_fig(filename, graphpath)

# Função para gerar histogramas
def generate_histplot(df, x, hue, title, filename, graphpath, bins=20):
    plt.figure(figsize=FIGSIZE)
    try:
        sns.histplot(df.compute(), x=x, hue=hue, bins=bins, palette=PALETTE, kde=True)
    except ValueError as e:
        print(f"Erro ao gerar histograma: {e}")
        return
    plt.title(title)
    save_fig(filename, graphpath)

# Função para gerar gráficos KDE
def generate_kdeplot(df, x, hue, title, filename, graphpath):
    plt.figure(figsize=FIGSIZE)
    try:
        sns.kdeplot(df.compute(), x=x, hue=hue, fill=True, palette=PALETTE)
    except ValueError as e:
        print(f"Erro ao gerar gráfico KDE: {e}")
        return
    plt.title(title)
    save_fig(filename, graphpath)

# Função para gerar regressões polinomiais
def generate_polynomial_regression(df, x, y, degree, title, filename, graphpath):
    plt.figure(figsize=FIGSIZE)
    try:
        sns.regplot(x=x, y=y, data=df.compute(), order=degree, scatter_kws={'s': 5}, line_kws={'color': 'red'})
    except ValueError as e:
        print(f"Erro ao gerar regressão polinomial: {e}")
        return
    plt.title(title)
    save_fig(filename, graphpath)

# Função para gerar boxplots
def generate_boxplot(df, x, y, hue, title, filename, graphpath):
    plt.figure(figsize=FIGSIZE)
    try:
        sns.boxplot(x=x, y=y, hue=hue, data=df.compute(), palette=PALETTE)
    except ValueError as e:
        print(f"Erro ao gerar boxplot: {e}")
        return
    plt.title(title)
    save_fig(filename, graphpath)

# Função para gerar gráficos de barras
def generate_barplot(df, x, y, hue, title, filename, graphpath):
    plt.figure(figsize=FIGSIZE)
    try:
        sns.barplot(x=x, y=y, hue=hue, data=df.compute(), palette=PALETTE)
    except ValueError as e:
        print(f"Erro ao gerar gráfico de barras: {e}")
        return
    plt.title(title)
    save_fig(filename, graphpath)

# Função para realizar bootstrap paralelizado com Numba
@jit(nopython=True, parallel=True)
def bootstrap_jit(data, boots=1000):
    bs_data = np.empty(boots)
    for b in prange(boots):
        total = 0
        for s in range(data.shape[0]):
            total += data[np.random.randint(0, data.shape[0])]
        bs_data[b] = total / data.shape[0]
    return bs_data

# Função para calcular intervalos de confiança via bootstrap
def bootstrap_confidence_intervals(data, boots=1000, alpha=0.05):
    boot_means = bootstrap_jit(data, boots)
    lower = np.percentile(boot_means, 100 * alpha / 2)
    upper = np.percentile(boot_means, 100 * (1 - alpha / 2))
    return lower, upper

# Função para gerar sumário estatístico detalhado e salvar em CSV
def generate_statistical_summary_spss(df, summarypath):
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    summary = df[numeric_columns].describe().compute().T
    summary['variance'] = df[numeric_columns].var().compute()
    summary['skewness'] = [skew(df[col].compute()) for col in numeric_columns]
    summary['kurtosis'] = [kurtosis(df[col].compute()) for col in numeric_columns]
    summary.to_csv(os.path.join(summarypath, "summary_spss.csv"))
    print(f"Sumário estatístico salvo em: {summarypath}")

# Função para gerar gráfico preditivo usando regressão linear
def generate_predictive_plot(df, x, y, title, filename, graphpath):
    plt.figure(figsize=FIGSIZE)

    # Dividir os dados em treino e teste
    X = df[x].compute().values.reshape(-1, 1)
    Y = df[y].compute().values
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    # Treinar o modelo de regressão linear
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predições
    y_pred = model.predict(X_test)

    # Criar gráfico de dispersão e linha de predição
    plt.scatter(X_test, y_test, color=PALETTE[1], s=5, label="Dados Reais")
    plt.plot(X_test, y_pred, color=PALETTE[0], label="Linha de Regressão", linewidth=2)

    # Adicionar título e informações
    plt.title(f"{title} (MSE: {mean_squared_error(y_test, y_pred):.4f})")
    plt.xlabel(x)
    plt.ylabel(y)
    plt.legend()

    save_fig(filename, graphpath)

# Exemplo de uso com dados simulados
if __name__ == "__main__":
    # Criar diretórios para salvar gráficos e sumários
    os.makedirs(GRAPHPATH, exist_ok=True)
    os.makedirs(SUMMARYPATH, exist_ok=True)

    # Carregar dados CSV
    CSV_FILEPATH = "https://docs.google.com/spreadsheets/d/15wgJVAwmunPxMaRzOjxXjaVm92eFNV5zbjAB8xKDffc/export?format=csv&gid=2134230752"
    df = load_data(CSV_FILEPATH)

    if df is not None:
        # Codificar colunas categóricas, se existirem
        if 'Sexo' in df.columns:
            le_gender = LabelEncoder()
            df = df.assign(Gender_Num=dd.from_array(le_gender.fit_transform(df['Sexo'].compute())))

        if 'DiagnosticoFormal' in df.columns:
            le_diagnosis = LabelEncoder()
            df = df.assign(Diagnosis_Num=dd.from_array(le_diagnosis.fit_transform(df['DiagnosticoFormal'].compute())))

        # Verificar as colunas antes de gerar gráficos
        if 'Idade' in df.columns and 'Sexo' in df.columns and 'DiagnosticoFormal' in df.columns:
            # Filtrar apenas para "Sim" e "Não" no eixo x do primeiro gráfico
            df_filtered = df[df['DiagnosticoFormal'].isin(['Sim', 'Nao'])]

            # Gerar gráficos
            generate_histplot(df, 'Idade', 'Sexo', 'Histograma: Idade por Gênero', 'hist_idade_genero.png', GRAPHPATH)

            generate_kdeplot(df, 'Idade', 'DiagnosticoFormal', 'KDE: Idade por Diagnóstico', 'kde_idade_diagnostico.png', GRAPHPATH)

            generate_kdeplot(df, 'Idade', 'Sexo', 'KDE: Idade por Gênero', 'kde_idade_genero.png', GRAPHPATH)

            generate_polynomial_regression(df, 'Idade', 'Diagnosis_Num', 3,
                                           'Regressão Polinomial (Idade vs Diagnóstico - Grau 3)',
                                           'polynomial_diagnosis_degree_3.png', GRAPHPATH)

            generate_boxplot(df, 'DiagnosticoFormal', 'Idade', 'Sexo',
                             'Boxplot: Idade por Gênero e Diagnóstico', 'boxplot_idade_diagnostico_genero.png', GRAPHPATH)

            generate_barplot(df, 'DiagnosticoFormal', 'Idade', 'Sexo',
                             'Barplot: Idade por Gênero e Diagnóstico', 'barplot_idade_diagnostico_genero.png', GRAPHPATH)

            generate_violinplot(df, 'DiagnosticoFormal', 'Idade', 'Sexo',
                                'Violin: Idade por Diagnóstico e Gênero', 'violin_idade_diagnostico_genero.png', GRAPHPATH)

            # Gerar gráfico preditivo (nono gráfico)
            if 'Idade' in df.columns and 'Pontuacao' in df.columns:
                generate_predictive_plot(df, 'Idade', 'Pontuacao',
                                         'Previsão de Pontuação com Base na Idade',
                                         'predictive_plot_idade_pontuacao.png', GRAPHPATH)

            # Gerar sumário estatístico
            generate_statistical_summary_spss(df, SUMMARYPATH)

            # Exemplo de bootstrap com dados simulados
            data = np.random.normal(0, 1, size=100)
            lower, upper = bootstrap_confidence_intervals(data, boots=1000)
            print(f"Intervalo de confiança via bootstrap: {lower:.4f}, {upper:.4f}")
        else:
            print("Colunas necessárias ('Idade', 'Sexo', 'DiagnosticoFormal') não estão presentes no dataframe.")

<ipython-input-28-50ea7f566c70>:50: UserWarning: The palette list has more values (8) than needed (2), which may not be intended.
  sns.histplot(df.compute(), x=x, hue=hue, bins=bins, palette=PALETTE, kde=True)
<ipython-input-28-50ea7f566c70>:61: UserWarning: The palette list has more values (8) than needed (3), which may not be intended.
  sns.kdeplot(df.compute(), x=x, hue=hue, fill=True, palette=PALETTE)
<ipython-input-28-50ea7f566c70>:61: UserWarning: The palette list has more values (8) than needed (2), which may not be intended.
  sns.kdeplot(df.compute(), x=x, hue=hue, fill=True, palette=PALETTE)
/usr/local/lib/python3.10/dist-packages/seaborn/regression.py:261: RankWarning: Polyfit may be poorly conditioned
  return np.polyval(np.polyfit(_x, _y, order), grid)
/usr/local/lib/python3.10/dist-packages/seaborn/regression.py:261: RankWarning: Polyfit may be poorly conditioned
  return np.polyval(np.polyfit(_x, _y, order), grid)
/usr/local/lib/python3.10/dist-packages/seaborn/regress

Sumário estatístico salvo em: /content/drive/MyDrive/sumarios
Intervalo de confiança via bootstrap: -0.3825, 0.0116
